## Outline

1. Import Packages

2. Define Functions

3. Run Functions to Train Models, Save Models, and Log Results

### 1. Import Packages

In [2]:
import pandas as pd
import numpy as np
import statistics
import pathlib
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from os.path import exists


from tqdm import tqdm

import tensorflow as tf
from tensorflow import keras
from keras.models import load_model
from tensorflow.keras import models, layers, backend, regularizers
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

from keras.models import Sequential
from keras import layers
from tensorflow.keras.optimizers import RMSprop

from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, classification_report

### 2. Define Functions

In [2]:
def generator(data, actual_close_prices, lookback, delay, min_index, max_index,
              shuffle=False, batch_size=128, step=6):
    
    '''
    ###################################################################
    Generates batches of data for the model to train on.  Saves space 
    in memory, which allows Deep Learning models to analyze big datasets.
    
    Generators will be run on the train and validation data.
    
    Inputs cleaned pricing data and outputs the generator.
    
    Adapted from: Deep Learning With Python, by Francis Chollet
    ###################################################################

    '''
    # Checks index
    if max_index is None:
        max_index = len(data) - delay - 1
    i = min_index + lookback
    
    # Gets samples and targets for each item in the batch
    while 1:
        if shuffle:
            rows = np.random.randint(
                min_index + lookback, max_index, size=batch_size)
        else:
            if i + batch_size >= max_index:
                i = min_index + lookback
            rows = np.arange(i, min(i + batch_size, max_index))
            i += len(rows)

        samples = np.zeros((len(rows),
                           lookback // step,
                           data.shape[-1]))
        targets = np.zeros((len(rows),))
        
        for j, row in enumerate(rows[:-2]):
            indices = range(rows[j] - lookback, rows[j], step)
            samples[j] = data[indices]
            #targets[j] = data[rows[j] + delay][1]
            
            # Calculate custom target
            beg = actual_close_prices[rows[j]]
            end = actual_close_prices[rows[j] + delay]
            value = (end-beg)/beg
            
            if value > 0:
                targets[j] = 1
            else:
                targets[j] = 0
                
        yield samples, targets
        
def train_validation_test_split_and_scaling(float_data, train_percent, val_percent):
    
    '''
    ###################################################################
    Gets the row numbers for train, validation, and test sets within the 
    array.  Scales the data in place based on the test data.
    
    Input is array of data with split percents.
    
    Output is the scaled data and the data point end rows.
    ###################################################################
    '''
    
    # Train, validation, test split
    n = len(float_data)

    train_data_end = int(n*train_percent) #0.7
    val_data_end = int(n* (train_percent + val_percent)) #0.9
    test_data_end = n

    # Feature Scaling
    mean = float_data[:train_data_end].mean(axis=0)
    float_data -= mean
    std = float_data[:train_data_end].std(axis=0)
    float_data /= std
    
    return float_data, train_data_end, val_data_end,test_data_end

def fit_model(train_gen, val_gen, val_steps, test_array, test_labels, verbose):
    
    '''
    ###################################################################
    Creates the LSTM model, compiles it, and trains it.  
    
    Inputs are the generators and test data and labels.
    
    Outputs the model and its history
    ###################################################################
    '''

    backend.clear_session()

    model = Sequential()

    model.add(layers.LSTM(128,
                         dropout=0.2,
                         recurrent_dropout=0.5,
                         return_sequences=True,
                         input_shape=(None, float_data.shape[-1])))

    model.add(layers.LSTM(128, activation='relu',
                         dropout=0.2,
                         recurrent_dropout=0.5))

    model.add(layers.Dense(1, activation = 'sigmoid'))

    model.compile(optimizer=RMSprop(), loss='binary_crossentropy',metrics = ['accuracy'])

    history = model.fit(train_gen,
                      shuffle=False,
                      steps_per_epoch=100,
                      epochs=40,
                      validation_data=val_gen,
                      validation_steps=val_steps,
                      verbose = verbose,
                      callbacks=[EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights = True)])

    return model, history

def plot_model_loss_accuracy(history):
    
    '''
    ###################################################################
    Plots Model Evaluation Metrics
    ###################################################################
    '''

    history_dict = history.history
    loss_values = history_dict['loss']
    val_loss_values = history_dict['val_loss']
    acc_values = history_dict['accuracy']
    val_acc_values = history_dict['val_accuracy']
    epochs = range(1, len(history_dict['accuracy']) + 1)

    plt.plot(epochs, loss_values, 'bo', label = 'Training loss')
    plt.plot(epochs, val_loss_values, 'b', label = 'Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

    plt.plot(epochs, acc_values, 'bo', label = 'Training accuracy')
    plt.plot(epochs, val_acc_values, 'b', label = 'Validation accuracy')
    plt.title('Training and validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()
    
    return

def save_model(delay, co, model):
    
    '''
    ###################################################################
    Saves the Model Locally
    ###################################################################
    '''

    str_delay = str(delay) + ' min'
    
    destination = pathlib.Path.cwd() / 'Models'/ str_delay
    if not destination.exists():
        destination.mkdir(parents=True, exist_ok=True)

    path = 'Models/'+ str_delay +'/'
    file_name = path + co + ' ' + str_delay + ".h5"

    model.save(file_name, save_format="tf")
    
    return

def load_a_model(delay, co):
    
    '''
    ###################################################################
    Loads in the model from local storage
    ###################################################################
    
    '''
    str_delay = str(delay) + ' min'
    path = 'Models/'+ str_delay +'/'
    file_name = path + co + ' ' + str_delay + ".h5"

    loaded_model = load_model(file_name)
    
    return loaded_model

def generator_and_label_provider(float_data, actual_close_prices, lookback,step, delay, batch_size, train_data_end,val_data_end):

    '''
    ###################################################################
    A versatile function to prepare data for model training and 
    evaluation.  Creates the train and validation generators as the
    testing data and labels. 
    ###################################################################
    
    '''
    
    # Create Generators
    train_gen = generator(float_data,actual_close_prices,
                          lookback=lookback,
                          delay=delay,
                          min_index=0,
                          max_index=train_data_end,
                          shuffle=True,
                          step=step,
                          batch_size=batch_size)
    val_gen = generator(float_data, actual_close_prices,
                        lookback=lookback,
                        delay=delay,
                        min_index=train_data_end,
                        max_index=val_data_end,
                        step=step,
                        batch_size=batch_size)

    val_steps = (val_data_end - train_data_end - lookback) // step // batch_size

    #test_steps = (len(float_data) - val_data_end - lookback) // step // batch_size

    # Get test data and test labels
    test_data = float_data[val_data_end:]
    test_prices = actual_close_prices[val_data_end:]

    test_data_list = []
    test_values_list = []
    test_hours_list = []
    test_minutes_list = []
    test_weekdays_list = []
    
    # Saves test data in chunks and labels and misc data for evaluations and result logging
    for x in range(lookback, len(test_data) - delay, step):
        x_data = test_data[x-lookback:x]
        test_data_list.append(x_data)
        end_price = test_prices[x+delay]
        beg_price = test_prices[x]
        value_change = (end_price - beg_price) / beg_price
        test_values_list.append(value_change)

        test_hours_list.append(hours[x])
        test_minutes_list.append(minutes[x])
        test_weekdays_list.append(weekdays[x])

    test_array = np.asarray(test_data_list)

    # Converts percent values to binary labels
    test_labels = []

    for label in test_values_list:
        if label > 0:
            test_labels.append(1)
        else:
            test_labels.append(0)

    test_labels = np.asarray(test_labels)

    # Counts class instances, for class balance checks
    zero_count = 0
    one_count = 0

    for label in test_labels:
        if label == 0:
            zero_count += 1
        else:
            one_count += 1

    #print("Zeros: ", zero_count)
    #print("Ones: ", one_count)
    
    return train_gen, val_gen, val_steps, zero_count, one_count, test_array, test_labels, test_values_list, test_hours_list, test_minutes_list, test_weekdays_list

def log_result_by_delay(model, test_array, test_labels, test_values_list, delay):
    
    '''
    ###################################################################
    Logs the results by delay period, which is 30 minutes for this project
    but different delay periods could be experimented wtih.  For the given
    model, the fucntion calculates ROC Score, Accuracy, investment returns,
    passive returns, and a classification matrix.  The function ends by
    loggin the results into a spreadsheet.
    ###################################################################
    '''
    
    ########################### Run Metrics ############################################
    test_loss, test_acc = model.evaluate(test_array, test_labels, verbose=0)

    y_pred = model.predict(test_array)
    y_pred_list = []

    # Convert probabilities to binary prediction
    for item in y_pred:
        if item > .50: #.50
            y_pred_list.append(1)
        else:
            y_pred_list.append(0)

    # For companies that the model predicts as postive, add the % return to a list and average
    investments = []
    for x in range(len(test_values_list)):
        if list(y_pred_list)[x] == 1:
            investments.append(list(test_values_list)[x])

    # Compare average returns given by the model, compared to average returns overall
    try:
        strat_avg = statistics.mean(investments)
    except:
        strat_avg = 0

    try:
        passive_avg = statistics.mean(test_values_list)
    except:
        passive_avg = 0

    '''
    ########################################################################################################################
    ROC Score
    ########################################################################################################################
    '''
    try:
        roc_score = roc_auc_score(test_labels, y_pred)
    except:
        roc_score = 0
        
    print('ROC Score:', roc_score)
    print('Strategy Returns', round(strat_avg,5)*100,"%")
    print('Passive Returns:', round(passive_avg,5)*100,"%")

    '''
    ########################################################################################################################
    Classification Report
    ########################################################################################################################
    '''
    #print("________________________________________________________________________")
    #print("Classification Report")
    #print(classification_report(test_labels, y_pred_list, zero_division=0))

    '''
    ########################################################################################################################
    Plot Confusion Matrix
    ########################################################################################################################
    '''
    cm = confusion_matrix(test_labels, y_pred_list)

    try:
        tp = cm[1][1]
        fp = cm[0][1]
        tn = cm[0][0]
        fn = cm[1][0]
    except:
        tp = 0
        fp = 0
        tn = 0
        fn = 0

    ######################################## Initiate Log ##################################

    # See if folder exists, if not, create it
    destination = pathlib.Path.cwd() / 'Logs'/ 'Result Logs' / 'By Delay'
    if not destination.exists():
        destination.mkdir(parents=True, exist_ok=True)


    # See if file exists, if not create it    
    file_name = 'Result Log by Delay - '+str(delay)+' min.csv'
    file = pathlib.Path.cwd() / 'Logs'/ 'Result Logs' / 'By Delay' / file_name

    if not file.is_file():
        cols = ['Ticker', 'Zero_count', 'One_count', 'Accuracy', 'ROC Score', 'TP', 'FP','TN','FN', 'Strat Return', 'Passive Return']

        result_log = pd.DataFrame(columns=cols)
        result_log.to_csv(file, index=False)

    ######################################### Add to File #####################################

    #open file
    result_log = pd.read_csv(file)

    # create new row
    cols = ['Ticker', 'Zero_count', 'One_count', 'Accuracy', 'ROC Score', 'TP', 'FP','TN','FN', 'Strat Return', 'Passive Return']

    new_row = pd.DataFrame([[co, zero_count, one_count,test_acc,roc_score,tp,fp,tn,fn,strat_avg,passive_avg]], columns=cols)

    # append row to file
    result_log = result_log.append(new_row, ignore_index=True)

    # Save
    result_log.to_csv(file, index=False)
    
    return

def log_result_by_window(model, delay, test_array, test_labels, test_values_list, test_hours_list, test_minutes_list, test_weekdays_list):

    '''
    ###################################################################
    This function is similar to the one above except it logs results on
    a more granular basis.  This function breaks the test data into 
    trading windows.  The trading week is broken down into Day, hour, 
    minute (0 or 30), windows.  So each week has 70 windows.  The test 
    data represents several weeks, so this function evaluates model 
    performance on a granular basis by averaging or calculating metrics
    on a subset of the data - each window.
    ###################################################################
    '''
    
    ######################################## Initiate Log ##################################

    # See if folder exists, if not, create it
    destination = pathlib.Path.cwd() / 'Logs'/ 'Result Logs' / 'By Window'
    if not destination.exists():
        destination.mkdir(parents=True, exist_ok=True)


    # See if file exists, if not create it    
    file_name = 'Result Log by Window - '+str(delay)+' min.csv'
    file = pathlib.Path.cwd() / 'Logs'/ 'Result Logs' / 'By Window' / file_name

    if not file.is_file():
        cols = ['Ticker', 'Weekday','Hour','Minute', 'ROC Score', 'TP', 'FP','TN','FN', 'Strat Return', 'Passive Return']

        result_log = pd.DataFrame(columns=cols)
        result_log.to_csv(file, index=False)

    ######################################### Add to File #####################################

    #open file
    result_log = pd.read_csv(file)

    ######################################## Create Result DF ##################################
    pred_df = pd.DataFrame()

    y_pred = model.predict(test_array)
    y_pred_list = [x[0] for x in y_pred]

    pred_df['pred'] = y_pred_list
    pred_label = [1 if x >.50 else 0 for x in y_pred_list]
    pred_df['predicted label'] = pred_label
    pred_df['actual label'] = test_labels
    pred_df['% return'] = test_values_list
    pred_df['hour'] = test_hours_list
    pred_df['minute'] = test_minutes_list
    pred_df['weekday'] = test_weekdays_list

    ###################################### Group DF by each Period ##################################

    for group in pred_df.groupby(['hour','minute','weekday']):

        hour = group[0][0]
        minute = group[0][1]
        weekday = group[0][2]

        try:
            roc_score = roc_auc_score(group[1]['actual label'], group[1]['predicted label'])
        except:
            roc_score = 0

        cm = confusion_matrix(group[1]['actual label'], group[1]['predicted label'])

        try:
            tp = cm[1][1]
            fp = cm[0][1]
            tn = cm[0][0]
            fn = cm[1][0]
        except:
            tp = 0
            fp = 0
            tn = 0
            fn = 0
            

        #print(len(group[1]['predicted label']))

        # For companies that the model predicts as postive, add the % return to a list and average
        investments = []
        for x in range(len(group[1]['% return'])):
            if list(group[1]['predicted label'])[x] == 1:
                investments.append(list(group[1]['% return'])[x])

        # Compare average returns given by the model, compared to average returns overall
        try:
            strat_avg = statistics.mean(investments)
        except:
            strat_avg = 0

        try:
            passive_avg = statistics.mean(group[1]['% return'])
        except:
            passive_avg = 0

        strat_returns = strat_avg
        passive_returns = passive_avg

        # create new row
        cols = ['Ticker', 'Weekday','Hour','Minute', 'ROC Score', 'TP', 'FP','TN','FN', 'Strat Return', 'Passive Return']

        new_row = pd.DataFrame([[co,weekday, hour, minute ,roc_score,tp,fp,tn,fn,strat_avg,passive_avg]], columns=cols)

        # append row to file
        result_log = result_log.append(new_row, ignore_index=True)


    # Save
    result_log.to_csv(file, index=False)
    
    return


### 3. Run Functions to Train Models, Save Models, and Log Results

## 30 Mins

In [3]:
###############################################################################################
# Import List of Stocks and Proxies
###############################################################################################

# List of Stocks
#CoList = pd.read_excel('Input Files/List of ETFs.xlsx',sheet_name='Low_Missing')
CoList = pd.read_excel('Input Files/List of ETFs.xlsx',sheet_name='Demo')

CoList = CoList['Symbol']

# List of Proxies
proxies = pd.read_excel('Input Files/List of ETFs.xlsx',sheet_name='Proxies')

proxies = proxies['Symbol']

# List of Market Holidays
holiday_list = list(pd.read_excel('Input Files/Stock Market Holidays.xlsx')['Date'])
holiday_list = [holiday.strftime('%Y-%m-%d') for holiday in holiday_list]

################## Set Interval #######################
# 1min, 5min, 15min, 30min, 60min
minute_interval = '1min'



###############################################################################################
# Set Parameters
###############################################################################################
delay = 30

lookback = 390*3
step = 30
batch_size = 128

###############################################################################################
# Run Functions to Train all the models, save them, and log the results
###############################################################################################
cos_added = 0
cos_existing = 0

for co in CoList:
    
    print('+++++++++++++++++++++++++++++++++++++++')
    print('Processing:',co)
    
    # Make sure model doesn't already exist, skip if so
    str_delay = str(delay) + ' min'
    path = 'Models/'+ str_delay +'/'
    file = path + co + ' ' + str_delay + ".h5"

    if not exists(file):
        
        ######################### Load in the dataset #############################################
        
        file = 'AlphaVantage Data/1min/6. Cleaned Combined Data Stationarity/1min '+co+' cleaned_combined_stationary.csv'

        df = pd.read_csv(file)

        df = df.drop(columns=['time','datetime','short_date', 'hour_minute'])

        hours = df['hour']
        minutes = df['minute']
        weekdays = df['weekday']

        actual_close_prices = np.asarray(df['close_prices'])
        float_data = df.to_numpy()
        
        ################################## Split into Train, Validation, and Test data, and scale features ##################
        # 65% train data, 15% validation data, 20% Test Data
        float_data, train_data_end, val_data_end,test_data_end = train_validation_test_split_and_scaling(float_data, .70, .20)
        
        ############################# Create Data Generators and Make Labels and Testing Aids ######################
        train_gen, val_gen, val_steps, zero_count, one_count, test_array, test_labels, test_values_list, test_hours_list, test_minutes_list, test_weekdays_list =  generator_and_label_provider(float_data, actual_close_prices, lookback,step, delay, batch_size, train_data_end,val_data_end)
        
        print("Training the Model")
        ####################################### Fit the Model #############################################
        
        # Enables multiple tries
        third_met = False
        third_counter = 0
        
        while third_met == False:
            
            model, history = fit_model(train_gen, val_gen, val_steps, test_array, test_labels, 0)
            
            y_pred = model.predict(test_array)
            y_pred_list = []

            # Convert probabilities to binary prediction
            for item in y_pred:
                if item > .50: #.50
                    y_pred_list.append(1)
                else:
                    y_pred_list.append(0)
            
            cm = confusion_matrix(test_labels, y_pred_list)
            try:
                tp = cm[1][1]
                fp = cm[0][1]
                tn = cm[0][0]
                fn = cm[1][0]
            except:
                tp = 0
                fp = 0
                tn = 0
                fn = 0
                
            # Try multiple times unless model makes semi-balanced prediction, otherwise some models predict < 1% for some classes
            if (tp+fp)/(tp+fp+tn+fn) > .30 and (tn+fn)/(tp+fn+tn+fn) > .30:
                third_met = True
                print("Third Met")
                
            elif third_counter > 3:
                third_met = True
                print("Model Tried More than three times...advancing")
            else:
                third_counter += 1
                print("Third Not Met")
        

        ################################ Save the Model and Load it back in for testing ######################
        save_model(delay, co, model)
        model = load_a_model(delay, co)
        
        print("Logging Result")
        ################################## Log the Result by each delay period ###############################
        log_result_by_delay(model,test_array, test_labels, test_values_list, delay)
        
        ################################## Log Result by each trading window for each delay period ####################
        log_result_by_window(model, delay, test_array, test_labels, test_values_list, test_hours_list, test_minutes_list, test_weekdays_list)
        
        cos_added += 1
        
    else:
        cos_existing += 1
        continue
        
print('All Finished')
print('Cos added:', cos_added)
print('Cos Already Existing:', cos_existing)
print('Total Cos in Folders:',cos_added + cos_existing)

+++++++++++++++++++++++++++++++++++++++
Processing: AGG
+++++++++++++++++++++++++++++++++++++++
Processing: AMLP
+++++++++++++++++++++++++++++++++++++++
Processing: BND
+++++++++++++++++++++++++++++++++++++++
Processing: DIA
+++++++++++++++++++++++++++++++++++++++
Processing: EEM
+++++++++++++++++++++++++++++++++++++++
Processing: EFA
+++++++++++++++++++++++++++++++++++++++
Processing: EWJ
+++++++++++++++++++++++++++++++++++++++
Processing: EWZ
+++++++++++++++++++++++++++++++++++++++
Processing: FXI
+++++++++++++++++++++++++++++++++++++++
Processing: GDX
+++++++++++++++++++++++++++++++++++++++
Processing: GDXJ
+++++++++++++++++++++++++++++++++++++++
Processing: GLD
+++++++++++++++++++++++++++++++++++++++
Processing: HYG
+++++++++++++++++++++++++++++++++++++++
Processing: IAU
+++++++++++++++++++++++++++++++++++++++
Processing: IEFA
+++++++++++++++++++++++++++++++++++++++
Processing: IEMG
+++++++++++++++++++++++++++++++++++++++
Processing: IJR
+++++++++++++++++++++++++++++++++++++++
Proc